# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

# Eager mode on

In [2]:
tf.enable_eager_execution()

# Checkpoint function

In [3]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [4]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [5]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [6]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def sigmoid() :
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

# Create model (class version)

In [7]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(sigmoid())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

# Create model (function version)

In [8]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(sigmoid())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [9]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

# Define model & optimizer & writer

In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.contrib.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    global_step = tf.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.3214, train_loss: 2.23722649, train_accuracy: 0.1641, test_Accuracy: 0.1010
Epoch: [ 0] [    1/  468] time: 0.4400, train_loss: 2.28909516, train_accuracy: 0.1172, test_Accuracy: 0.1135
Epoch: [ 0] [    2/  468] time: 0.5412, train_loss: 2.30295420, train_accuracy: 0.0859, test_Accuracy: 0.1135
Epoch: [ 0] [    3/  468] time: 0.6293, train_loss: 2.26018047, train_accuracy: 0.1484, test_Accuracy: 0.1135
Epoch: [ 0] [    4/  468] time: 0.7447, train_loss: 2.28151369, train_accuracy: 0.1172, test_Accuracy: 0.1136
Epoch: [ 0] [    5/  468] time: 0.8545, train_loss: 2.26245213, train_accuracy: 0.2109, test_Accuracy: 0.2517
Epoch: [ 0] [    6/  468] time: 0.9439, train_loss: 2.22125435, train_accuracy: 0.1719, test_Accuracy: 0.1654
Epoch: [ 0] [    7/  468] time: 1.0468, train_loss: 2.25816059, train_accuracy: 0.2031, test_Accuracy: 0.2150
Epoch: [ 0] [    8/  468] time: 1.1519,

Epoch: [ 0] [   76/  468] time: 7.7875, train_loss: 0.97948754, train_accuracy: 0.7500, test_Accuracy: 0.7898
Epoch: [ 0] [   77/  468] time: 7.8770, train_loss: 0.81453854, train_accuracy: 0.8516, test_Accuracy: 0.7857
Epoch: [ 0] [   78/  468] time: 7.9662, train_loss: 0.87466240, train_accuracy: 0.7500, test_Accuracy: 0.7879
Epoch: [ 0] [   79/  468] time: 8.0544, train_loss: 0.82989830, train_accuracy: 0.8438, test_Accuracy: 0.7888
Epoch: [ 0] [   80/  468] time: 8.1536, train_loss: 0.92940915, train_accuracy: 0.7578, test_Accuracy: 0.7937
Epoch: [ 0] [   81/  468] time: 8.2550, train_loss: 0.86642623, train_accuracy: 0.7812, test_Accuracy: 0.8009
Epoch: [ 0] [   82/  468] time: 8.3475, train_loss: 0.94328272, train_accuracy: 0.8047, test_Accuracy: 0.8146
Epoch: [ 0] [   83/  468] time: 8.4356, train_loss: 0.80263299, train_accuracy: 0.8594, test_Accuracy: 0.8231
Epoch: [ 0] [   84/  468] time: 8.5233, train_loss: 0.77890849, train_accuracy: 0.8281, test_Accuracy: 0.8242
Epoch: [ 0

Epoch: [ 0] [  151/  468] time: 14.7386, train_loss: 0.52600503, train_accuracy: 0.8750, test_Accuracy: 0.8723
Epoch: [ 0] [  152/  468] time: 14.8294, train_loss: 0.45337841, train_accuracy: 0.8984, test_Accuracy: 0.8696
Epoch: [ 0] [  153/  468] time: 14.9179, train_loss: 0.42315960, train_accuracy: 0.9141, test_Accuracy: 0.8695
Epoch: [ 0] [  154/  468] time: 15.0111, train_loss: 0.41338128, train_accuracy: 0.9062, test_Accuracy: 0.8707
Epoch: [ 0] [  155/  468] time: 15.1031, train_loss: 0.50799429, train_accuracy: 0.8359, test_Accuracy: 0.8753
Epoch: [ 0] [  156/  468] time: 15.1913, train_loss: 0.49137384, train_accuracy: 0.8906, test_Accuracy: 0.8784
Epoch: [ 0] [  157/  468] time: 15.2799, train_loss: 0.41322923, train_accuracy: 0.8750, test_Accuracy: 0.8796
Epoch: [ 0] [  158/  468] time: 15.3736, train_loss: 0.46074125, train_accuracy: 0.8594, test_Accuracy: 0.8799
Epoch: [ 0] [  159/  468] time: 15.4692, train_loss: 0.39120460, train_accuracy: 0.9375, test_Accuracy: 0.8784
E

Epoch: [ 0] [  226/  468] time: 21.6090, train_loss: 0.35609168, train_accuracy: 0.8984, test_Accuracy: 0.9031
Epoch: [ 0] [  227/  468] time: 21.6981, train_loss: 0.32984114, train_accuracy: 0.9453, test_Accuracy: 0.9039
Epoch: [ 0] [  228/  468] time: 21.7895, train_loss: 0.44402510, train_accuracy: 0.8594, test_Accuracy: 0.9038
Epoch: [ 0] [  229/  468] time: 21.8872, train_loss: 0.37383503, train_accuracy: 0.8984, test_Accuracy: 0.9021
Epoch: [ 0] [  230/  468] time: 21.9783, train_loss: 0.26115927, train_accuracy: 0.9375, test_Accuracy: 0.9031
Epoch: [ 0] [  231/  468] time: 22.0693, train_loss: 0.41544580, train_accuracy: 0.8828, test_Accuracy: 0.9012
Epoch: [ 0] [  232/  468] time: 22.1633, train_loss: 0.37542719, train_accuracy: 0.8828, test_Accuracy: 0.8991
Epoch: [ 0] [  233/  468] time: 22.2576, train_loss: 0.29735994, train_accuracy: 0.9062, test_Accuracy: 0.8992
Epoch: [ 0] [  234/  468] time: 22.3474, train_loss: 0.35634488, train_accuracy: 0.8984, test_Accuracy: 0.8994
E

Epoch: [ 0] [  301/  468] time: 28.5559, train_loss: 0.15861154, train_accuracy: 0.9766, test_Accuracy: 0.9103
Epoch: [ 0] [  302/  468] time: 28.6478, train_loss: 0.22605327, train_accuracy: 0.9375, test_Accuracy: 0.9098
Epoch: [ 0] [  303/  468] time: 28.7412, train_loss: 0.39532244, train_accuracy: 0.8828, test_Accuracy: 0.9103
Epoch: [ 0] [  304/  468] time: 28.8278, train_loss: 0.37956220, train_accuracy: 0.8906, test_Accuracy: 0.9120
Epoch: [ 0] [  305/  468] time: 28.9128, train_loss: 0.35492954, train_accuracy: 0.8906, test_Accuracy: 0.9120
Epoch: [ 0] [  306/  468] time: 28.9992, train_loss: 0.23030055, train_accuracy: 0.9375, test_Accuracy: 0.9123
Epoch: [ 0] [  307/  468] time: 29.0873, train_loss: 0.28519267, train_accuracy: 0.9375, test_Accuracy: 0.9131
Epoch: [ 0] [  308/  468] time: 29.1702, train_loss: 0.44649100, train_accuracy: 0.8594, test_Accuracy: 0.9133
Epoch: [ 0] [  309/  468] time: 29.2606, train_loss: 0.27539647, train_accuracy: 0.9062, test_Accuracy: 0.9133
E

Epoch: [ 0] [  376/  468] time: 35.4512, train_loss: 0.15606324, train_accuracy: 0.9688, test_Accuracy: 0.9180
Epoch: [ 0] [  377/  468] time: 35.5446, train_loss: 0.41055191, train_accuracy: 0.8828, test_Accuracy: 0.9189
Epoch: [ 0] [  378/  468] time: 35.6386, train_loss: 0.32453680, train_accuracy: 0.9219, test_Accuracy: 0.9200
Epoch: [ 0] [  379/  468] time: 35.7337, train_loss: 0.23024115, train_accuracy: 0.9375, test_Accuracy: 0.9200
Epoch: [ 0] [  380/  468] time: 35.8236, train_loss: 0.28748322, train_accuracy: 0.9375, test_Accuracy: 0.9203
Epoch: [ 0] [  381/  468] time: 35.9163, train_loss: 0.35743976, train_accuracy: 0.8828, test_Accuracy: 0.9197
Epoch: [ 0] [  382/  468] time: 36.0071, train_loss: 0.20572484, train_accuracy: 0.9297, test_Accuracy: 0.9207
Epoch: [ 0] [  383/  468] time: 36.0997, train_loss: 0.27861348, train_accuracy: 0.9297, test_Accuracy: 0.9201
Epoch: [ 0] [  384/  468] time: 36.1920, train_loss: 0.24169070, train_accuracy: 0.9297, test_Accuracy: 0.9203
E

Epoch: [ 0] [  451/  468] time: 42.3697, train_loss: 0.25468284, train_accuracy: 0.9062, test_Accuracy: 0.9222
Epoch: [ 0] [  452/  468] time: 42.4626, train_loss: 0.15882659, train_accuracy: 0.9688, test_Accuracy: 0.9224
Epoch: [ 0] [  453/  468] time: 42.5534, train_loss: 0.29443750, train_accuracy: 0.9375, test_Accuracy: 0.9219
Epoch: [ 0] [  454/  468] time: 42.6466, train_loss: 0.25797576, train_accuracy: 0.9219, test_Accuracy: 0.9213
Epoch: [ 0] [  455/  468] time: 42.7405, train_loss: 0.31461453, train_accuracy: 0.8984, test_Accuracy: 0.9222
Epoch: [ 0] [  456/  468] time: 42.8353, train_loss: 0.29421049, train_accuracy: 0.8984, test_Accuracy: 0.9221
Epoch: [ 0] [  457/  468] time: 42.9286, train_loss: 0.33905104, train_accuracy: 0.9062, test_Accuracy: 0.9228
Epoch: [ 0] [  458/  468] time: 43.0237, train_loss: 0.23832510, train_accuracy: 0.9531, test_Accuracy: 0.9238
Epoch: [ 0] [  459/  468] time: 43.1206, train_loss: 0.30897641, train_accuracy: 0.8984, test_Accuracy: 0.9241
E

# Test accuracy : 92.41 %